In [1]:
import pandas as pd
from pymongo import MongoClient
import json

In [4]:
client = MongoClient('localhost', 27017)
db = client.db
comments = db.comments
companies = db.companylist

In [5]:
commentdf = pd.DataFrame(comments.find({'created_utc': {'$exists': 'true'}}))
companydf = pd.DataFrame(companies.find())

In [6]:
def get_len(list):
    return len(list)

In [7]:
commentdf['size'] = commentdf['stocks'].apply(get_len)

In [8]:
stockdf = commentdf.loc[commentdf['size'] != 0]

In [31]:
from sklearn.preprocessing import MultiLabelBinarizer
s = stockdf['stocks']
mlb = MultiLabelBinarizer()
onehotdf = pd.DataFrame(mlb.fit_transform(s), columns = mlb.classes_, index = stockdf.index)

In [32]:
connectionsdict = onehotdf.sum(axis = 1).to_dict()
connectionsdict = {k:v for k, v in connectionsdict.items() if v != 1}

onehotdf = onehotdf[onehotdf.index.isin(connectionsdict.keys())]

In [33]:
take_or = lambda x, y: x if x >= y else y
onehotdf['BRK.A'] = onehotdf['BRK.A'].combine(onehotdf['BRK.B'], take_or)
onehotdf.drop(columns = ['BRK.A', 'BRK.B'], inplace = True)
onehotdf['GOOG'] = onehotdf['GOOG'].combine(onehotdf['GOOGL'], take_or)
onehotdf.drop(columns = ['GOOGL'], inplace = True)
onehotdf['C'] = onehotdf['BLW'].combine(onehotdf['C'], take_or)
onehotdf.drop(columns = ['BLW'], inplace = True)
onehotdf['BMY'] = onehotdf['BMY~'].combine(onehotdf['BMY'], take_or)
onehotdf.drop(columns = ['BMY~'], inplace = True)
onehotdf['JPM'] = onehotdf['JPM'].combine(onehotdf['CCF'], take_or)
onehotdf.drop(columns = ['CCF'], inplace = True)
onehotdf['JPM'] = onehotdf['JPM'].combine(onehotdf['JP'], take_or)
onehotdf.drop(columns = ['JP'], inplace = True)
onehotdf['CMCSA'] = onehotdf['CMCSA'].combine(onehotdf['CCZ'], take_or)
onehotdf.drop(columns = ['CCZ'], inplace = True)
onehotdf['F'] = onehotdf['F'].combine(onehotdf['FORD'], take_or)
onehotdf.drop(columns = ['FORD'], inplace = True)
onehotdf['HD'] = onehotdf['HD'].combine(onehotdf['HOME'], take_or)
onehotdf.drop(columns = ['HOME'], inplace = True)
onehotdf['HP'] = onehotdf['HP'].combine(onehotdf['HPE'], take_or)
onehotdf.drop(columns = ['HPE'], inplace = True)
onehotdf['HP'] = onehotdf['HP'].combine(onehotdf['HPQ'], take_or)
onehotdf.drop(columns = ['HPQ'], inplace = True)
onehotdf['IEX'] = onehotdf['IEX'].combine(onehotdf['IDEX'], take_or)
onehotdf.drop(columns = ['IDEX'], inplace = True)
onehotdf['LTRPA'] = onehotdf['LTRPA'].combine(onehotdf['LTRPB'], take_or)
onehotdf.drop(columns = ['LTRPB'], inplace = True)
onehotdf['RDS.A'] = onehotdf['RDS.A'].combine(onehotdf['RDS.B'], take_or)
onehotdf.drop(columns = ['RDS.B'], inplace = True)
onehotdf['SPY'] = onehotdf['SPY'].combine(onehotdf['SPXC'], take_or)
onehotdf.drop(columns = ['SPXC'], inplace = True)
onehotdf['STZ'] = onehotdf['STZ'].combine(onehotdf['STZ.B'], take_or)
onehotdf.drop(columns = ['STZ.B'], inplace = True)
onehotdf['TAP'] = onehotdf['TAP'].combine(onehotdf['TAP.A'], take_or)
onehotdf.drop(columns = ['TAP.A'], inplace = True)
onehotdf['CMCSA'] = onehotdf['CMCSA'].combine(onehotdf['UVV'], take_or)
onehotdf.drop(columns = ['UVV'], inplace = True)
onehotdf['WF'] = onehotdf['WF'].combine(onehotdf['WFC'], take_or)
onehotdf.drop(columns = ['WFC'], inplace = True)

In [37]:
nonMatrix = ['AA', 'ABC', 'ACH', 'ACT', 'ADP', 'ADS', 'AG', 'AGE', 'AGO', 'AHH', 'AIN', 'AIR', 'AIRT', 'AL', 'ALL', 'ALOT', 'AM', 'AN', 'ANY','AONE', 'AOS', 'AP', 'APG', 'API', 'AR', 'AT', 'AU', 'AUTO', 'BAH', 'BAM', 'BAND', 'BB', 'BBC', 'BBQ', 'BE', 'BEAT', 'BEN', 'BEST', 'BIG', 'BILL', 'BIT', 'BK', 'BKE', 'BLL', 'BLUE', 'BOOM', 'BR', 'BRC', 'BRO', 'BY', 'CALM', 'CARE', 'CBD', 'CBOE', 'CC', 'CCS', 'CD', 'CEO', 'CFA', 'CIA', 'COLD', 'COO', 'CORE', 'CPA', 'CRY', 'CS', 'CSPI', 'CUZ', 'DAIO', 'DARE', 'DDS', 'DE', 'DL', 'DMS', 'DNOW', 'DOC', 'DOOR', 'DS', 'DTE', 'DUO', 'EAST', 'EAT', 'EC', 'ED', 'EIX', 'ELSE', 'ERII', 'ES', 'ET', 'ETN', 'EVER', 'EW', 'EXP', 'EXPR', 'EYE', 'FAM', 'FANG', 'FARM', 'FAST', 'FAT', 'FDS', 'FE', 'FIVE', 'FIX', 'FL', 'FLOW', 'FLY', 'FNB', 'FOR', 'FR', 'FREE', 'FSD', 'FT', 'FTC', 'FUND', 'GAIN', 'GDP', 'GES', 'GF', 'GHM', 'GIM', 'GL', 'GLAD', 'GLG', 'GOLF', 'GT', 'HA', 'HALO', 'HAS', 'HBMD', 'HE', 'HEAR', 'HERD', 'HERO', 'HES', 'HI', 'HOL', 'HOPE', 'HR', 'ICBK', 'ICE', 'III', 'INFI', 'INFO', 'IP', 'IQ', 'IR', 'IRL', 'IROQ', 'IRS', 'IT', 'ITT', 'JAZZ', 'JG', 'JOB', 'JOBS', 'JOE', 'KEN', 'KIDS', 'KIM', 'LAKE', 'LAND', 'LE', 'LEE', 'LEG', 'LEO', 'LIFE', 'LIVE', 'LN', 'LOAN', 'LOW', 'LPTX', 'MAC', 'MAIN', 'MAN', 'MC', 'MD', 'MEN', 'MET', 'MIC', 'MIN', 'MIND', 'MMT', 'MN', 'MNDO', 'MOD', 'MODN', 'MT', 'MTB', 'MUH', 'NC', 'NEO', 'NEXT', 'NG', 'NGL', 'NICE', 'NICK', 'NK', 'NOW', 'NSA', 'NWS', 'NaN', 'OC', 'OFC', 'OI', 'OIBR.C', 'OLD', 'ON', 'ONE', 'OPBK', 'OR', 'OTLK', 'OUT', 'PBR', 'PBR.A', 'PEAK', 'PH', 'PHD', 'PLAN', 'PLUS', 'PM', 'POOL', 'POST', 'PPL', 'PPT', 'PRO', 'PRT', 'PS', 'PSA', 'PT', 'PUMP', 'QTWO', 'QUAD', 'RCKT', 'RE', 'REAL', 'RH', 'RICK', 'RIG', 'RILY', 'RING', 'RNA', 'ROAD', 'ROCK', 'ROLL', 'RUN', 'SALT', 'SB', 'SC', 'SD', 'SF', 'SGU', 'SHIP', 'SIRI', 'SJW', 'SKY', 'SMH', 'SNP', 'SO', 'SON', 'SPGI', 'ST', 'STAY', 'STEP', 'SUM', 'TD', 'TDA', 'TDS', 'TEAM', 'TECH', 'TELL', 'THC', 'THO', 'TIMB$', 'TISI', 'TM', 'TOWN', 'TR', 'TREE', 'TRUE', 'TTMI', 'TU', 'TURN', 'TV', 'TW', 'TX', 'TY', 'TYL', 'UE', 'UI', 'UN', 'USB', 'UTF', 'VC', 'VERY', 'WAT', 'WH', 'WIFI', 'WINS', 'WIRE', 'WM', 'WOOD', 'XYF', 'STAR          ']

In [38]:
onehotdf.drop(columns = nonMatrix, inplace = True)

In [60]:
companydict = onehotdf.sum(axis = 0).to_dict()
companydict = {k:v for k, v in companydict.items() if v < 25}

onehotdf.drop(columns = companydict.keys(), inplace= True)

In [62]:
onehotdf = onehotdf.T

In [63]:
from scipy import sparse

matrix = onehotdf.to_numpy()
sparsematrix = sparse.csr_matrix(matrix)

adjacencyframe = pd.DataFrame(sparsematrix.dot(onehotdf.T), columns = onehotdf.index)
adjacencyframe.set_index(onehotdf.index, inplace= True)

In [65]:
import numpy as np
adjacencyframe.values[[np.arange(adjacencyframe.shape[0])]*2] = 0

In [69]:
edgelist = adjacencyframe.stack().reset_index()

edgelist.drop(edgelist.loc[edgelist[0] == 0].index, inplace = True)
edgelist.reset_index(drop=True, inplace=True)
edgelist.columns = ['source', 'target', 'weight']

In [79]:
namesdf = pd.DataFrame(columns = ['id'])
namesdf['id'] = list(adjacencyframe.columns)

'''namesdf['weight'] = list(onehotdf.sum(axis = 1))
mergeframe = companydf[['Symbol', 'Industry']]
mergeframe = pd.merge(namesdf, mergeframe, left_on='id', right_on='Symbol', how = 'left')
mergeframe.drop_duplicates('id', ignore_index=True, inplace=True)
mergeframe.drop(columns = ['Symbol'], inplace=True)'''

In [164]:
jsondict = {}
jsondict['links'] = edgelist.to_dict('records')
jsondict['names'] = mergeframe.to_dict('records')

In [165]:
with open('data.json', 'w') as fp:
    json.dump(jsondict, fp)

Link max weight 1731
Link min weight 1
Edge max size 8846
edgee min size 25

In [1]:
import json

In [4]:
with open('data.json') as json_file: 
    data = json.load(json_file) 

In [9]:
set(list(data['nodes']['Industry'].values))

TypeError: list indices must be integers or slices, not str

In [20]:
industrylist = []

In [22]:
for i in range(len(data['nodes'])):
    industrylist.append(data['nodes'][i]['Industry'])

In [24]:
set(industrylist)

{'Advertising',
 'Aerospace',
 'Agricultural Chemicals',
 'Air Freight/Delivery Services',
 'Apparel',
 'Auto Manufacturing',
 'Automotive Aftermarket',
 'Beverages (Production/Distribution)',
 'Biotechnology: Biological Products (No Diagnostic Substances)',
 'Biotechnology: In Vitro & In Vivo Diagnostic Substances',
 'Broadcasting',
 'Business Services',
 'Catalog/Specialty Distribution',
 'Clothing/Shoe/Accessory Stores',
 'Commercial Banks',
 'Computer Communications Equipment',
 'Computer Manufacturing',
 'Computer Software: Prepackaged Software',
 'Computer Software: Programming, Data Processing',
 'Computer peripheral equipment',
 'Construction/Ag Equipment/Trucks',
 'Consumer Electronics/Appliances',
 'Consumer Electronics/Video Chains',
 'Department/Specialty Retail Stores',
 'EDP Services',
 'Electric Utilities: Central',
 'Electrical Products',
 'Electronic Components',
 'Electronics Distribution',
 'Farming/Seeds/Milling',
 'Finance: Consumer Services',
 'Fluid Controls',
 '

In [ ]:
{'Auto': {''}}

In [25]:
testdict = {"Auto": {"Auto Manufacturing", "Automotive Aftermarket", "Motor Vehicles"},
  "Travel": {"Hotels/Resorts", "Rental/Leasing Companies", "Marine Transportation", "Transportation Services"},
  "Food Distribution": {"Food Chains", "Beverages (Production/Distribution)", "Food Distributors", "Meat/Poultry/Fish", "Packaged Foods"},
  "Agriculture": {"Agricultural Chemicals", "Construction/Ag Equipment/Trucks", "Farming/Seeds/Milling", "Industrial Machinery/Components"},
  "Media": {"Movies/Entertainment", "Broadcasting", "Newspapers/Magazines", "Television Services", "Services-Misc. Amusement & Recreation"},
  "Tech": {"Advertising", "Computer Manufacturing",
    "Computer Software: Prepackaged Software",
    "Computer Software: Programming, Data Processing",
    "Computer peripheral equipment","Computer Communications Equipment",
     "Consumer Electronics/Appliances", "Consumer Electronics/Video Chains",
     "Semiconductors", "Radio And Television Broadcasting And Communications Equipment", 
     "Telecommunications Equipment", "EDP Services", "Electrical Products", "Electronic Components",
    "Electronics Distribution"},
  "Aerospace": {"Air Freight/Delivery Services", "Aerospace", "Military/Government/Technical"},
  "Stock Market Index": {"Stock Market Index"},
  "Pharm": {"Specialty Chemicals", "Industrial Specialties", "Major Pharmaceuticals",
        "Biotechnology: Biological Products (No Diagnostic Substances)", "Biotechnology: In Vitro & In Vivo Diagnostic Substances",
        "Medical Specialities",
        "Medical/Dental Instruments",
        "Medical/Nursing Services"},
  "Services": {"Restaurants", "Catalog/Specialty Distribution", "Other Consumer Services", "Trucking Freight/Courier Services"},
  "Goods": {"Shoe Manufacturing", "Apparel", "Ordnance And Accessories", "Package Goods/Cosmetics"},
  "Finance": {"Savings Institutions", "Business Services", "Commercial Banks", "Finance: Consumer Services", "Investment Bankers/Brokers/Service", "Major Banks"},
  "Real Estate": {"Real Estate Investment Trusts", "Real Estate", "Property-Casualty Insurers"},
  "Department Stores": {"RETAIL: Building Materials", "Clothing/Shoe/Accessory Stores", "Department/Specialty Retail Stores", 
        "Home Furnishings", "Other Specialty Stores"},
  "Utilities": {"Electric Utilities: Central", "Fluid Controls", "Power Generation"},
  "Oil": {"Integrated oil Companies", 
        "Oil & Gas Production",
        "Oilfield Services/Equipment","Precious Metals"}}

In [28]:
list(testdict.keys())

['Auto',
 'Travel',
 'Food distribution',
 'Agriculture',
 'Media',
 'Tech',
 'Aerospace',
 'Stock Market Index',
 'Pharm',
 'Services',
 'Goods',
 'Finance',
 'Real Estate',
 'Department Stores',
 'Utilities',
 'Oil']

In [ ]:
{'Auto': rgb(175, 214, 247),
'Travel': rgb(194, 182, 212),
'Food distribution': rgb(224, 217, 215),
 'Agriculture': rgb(180, 206, 179),
 'Media': rgb(228, 216, 185),
 'Tech': rgb(223, 190, 175),
 'Aerospace': rgb(139, 116, 105),
 'Stock Market Index': rgb(105, 125, 139),
 'Pharm': rgb(139, 105, 134),
 'Services': rgb(177, 140, 171),
 'Goods': rgb(177, 158, 140),
 'Finance': rgb(160, 218, 163),
 'Real Estate': rgb(164, 172, 206),
 'Department Stores': rgb(112, 106, 146),
 'Utilities': rgb(131, 125, 167),
 'Oil': rgb(125, 155, 167)}